In [16]:
!pip install folium

     |████████████████████████████████| 102kB 7.1MB/s ta 0:00:011


In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium as folium
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [28]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')
postalCodeList = []
boroughList = []
neighborhoodList = []

In [29]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [30]:
        toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df['PostalCode'] = toronto_df['PostalCode'].map(lambda x: x.rstrip('\n'))
toronto_df['Borough'] = toronto_df['Borough'].map(lambda x: x.rstrip('\n'))
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [31]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [33]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [43]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

In [44]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [64]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
#map_toronto

In [46]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [47]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [65]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
#map_toronto

In [49]:
CLIENT_ID = 'PVPEVSGTR5WZUDUA4JQQ3ZGCWYC1BBKV4XWYLUJYRDAZ54TD' # your Foursquare ID
CLIENT_SECRET = 'S3G2BSR32XEHYT1CUB02MGXM3Z1SQOCQYVRXRIFY1JBCLRC3' # your Foursquare Secret
VERSION = '2020015' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PVPEVSGTR5WZUDUA4JQQ3ZGCWYC1BBKV4XWYLUJYRDAZ54TD
CLIENT_SECRET:S3G2BSR32XEHYT1CUB02MGXM3Z1SQOCQYVRXRIFY1JBCLRC3


In [89]:
radius = 500
LIMIT = 50

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=PVPEVSGTR5WZUDUA4JQQ3ZGCWYC1BBKV4XWYLUJYRDAZ54TD&client_secret=S3G2BSR32XEHYT1CUB02MGXM3Z1SQOCQYVRXRIFY1JBCLRC3&v=20200815&ll=43.6534817,-79.3839347&radius=500&limit=50".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [90]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1950, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Poke Guys,43.654895,-79.385052,Poke Place


In [91]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,50,50,50,50,50,50
M4K,East Toronto,"The Danforth West, Riverdale",50,50,50,50,50,50
M4L,East Toronto,"India Bazaar, The Beaches West",50,50,50,50,50,50
M4M,East Toronto,Studio District,50,50,50,50,50,50
M4N,Central Toronto,Lawrence Park,50,50,50,50,50,50
M4P,Central Toronto,Davisville North,50,50,50,50,50,50
M4R,Central Toronto,"North Toronto West, Lawrence Park",50,50,50,50,50,50
M4S,Central Toronto,Davisville,50,50,50,50,50,50
M4T,Central Toronto,"Moore Park, Summerhill East",50,50,50,50,50,50


In [92]:
venues_df['VenueCategory'].unique()[:50]

array(['Neighborhood', 'Plaza', 'Sushi Restaurant', 'Breakfast Spot',
       'Poke Place', 'Bookstore', 'Bubble Tea Shop',
       'Monument / Landmark', 'Shopping Mall', 'Art Museum',
       'Cosmetics Shop', 'Office', 'Gym', 'Coffee Shop',
       'Fast Food Restaurant', 'Ramen Restaurant', 'Clothing Store',
       'Miscellaneous Shop', 'Concert Hall', 'Hotel', 'Electronics Store',
       'Department Store', 'Restaurant', 'Italian Restaurant',
       'Seafood Restaurant', 'American Restaurant',
       'Vegetarian / Vegan Restaurant', 'Theater',
       'Furniture / Home Store', 'Food Court', 'Bakery',
       'Toy / Game Store', 'Comic Shop', 'Bank', 'Tanning Salon', 'Bar',
       "Women's Store", 'New American Restaurant', 'Gastropub',
       'Modern European Restaurant', 'Steakhouse'], dtype=object)

In [93]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1950, 44)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,Gym,Hotel,Italian Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Neighborhood,New American Restaurant,Office,Plaza,Poke Place,Ramen Restaurant,Restaurant,Seafood Restaurant,Shopping Mall,Steakhouse,Sushi Restaurant,Tanning Salon,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Women's Store
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [94]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 44)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,Gym,Hotel,Italian Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Neighborhood,New American Restaurant,Office,Plaza,Poke Place,Ramen Restaurant,Restaurant,Seafood Restaurant,Shopping Mall,Steakhouse,Sushi Restaurant,Tanning Salon,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Women's Store
0,M4E,East Toronto,The Beaches,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
1,M4K,East Toronto,"The Danforth West, Riverdale",0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
3,M4M,East Toronto,Studio District,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
4,M4N,Central Toronto,Lawrence Park,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
5,M4P,Central Toronto,Davisville North,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
7,M4S,Central Toronto,Davisville,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.02,0.04,0.02,0.04,0.02,0.02,0.04,0.02,0.02,0.02


In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
#neighborhoods_venues_sorted

(39, 13)


In [114]:
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [115]:
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)

(39, 16)


In [116]:
toronto_merged.sort_values(["Cluster Labels"], inplace=True)

In [117]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
21,Downtown Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
22,Central Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
23,Central Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
24,Central Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
25,Downtown Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
26,Downtown Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
27,Downtown Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
20,Downtown Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store
28,Downtown Toronto,0,Clothing Store,Theater,Electronics Store,Steakhouse,Cosmetics Shop,Seafood Restaurant,Plaza,Women's Store,Fast Food Restaurant,Department Store


In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Many results of neighbourhood fall in cluster 1.